In [1]:
import datetime as datetime
import datetime 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
events = pd.read_csv("events.csv.gzip", compression='gzip')
clicks = pd.read_csv("clicks.csv.gzip", compression='gzip')
installs = pd.read_csv("installs.csv.gzip", compression='gzip')
#auctions = pd.read_csv("auctions.csv.gzip", compression='gzip')

pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/maximiliano/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Transformo a datetime la fecha y las posiciones de clicks en floats

In [3]:
installs['created'] = pd.to_datetime(installs['created'])
events['date'] = pd.to_datetime(events['date'])
clicks['created'] = pd.to_datetime(clicks['created'])
#auctions['date'] = pd.to_datetime(auctions['date'])


clicks['touchX_float'] = clicks['touchX'].astype('float64')
clicks['touchY_float'] = clicks['touchY'].astype('float64')

In [4]:
installs['date_seconds_level'] = installs['created'].astype('datetime64[s]')
events['date_seconds_level'] = events['date'].astype('datetime64[s]')

### Agrego diferencias entre installs de la misma aplicacion para el mismo usuario (para borrar posibles errores de la tracking platform luego)

In [5]:
installs['seconds_to_prev_install'] = installs.sort_values(['ref_hash', 'created'], ascending=True).groupby(['ref_hash', 'application_id'])['created'].diff()
installs['seconds_to_prev_install'] = installs['seconds_to_prev_install']/np.timedelta64(1, 's')

### Elimino de installs las instalaciones con menos de un segundo de diferencia para el mismo usuario y misma applicacion (las suponemos error de tracking platform) (22077 installs menos)

In [6]:
installs = installs.loc[(installs['seconds_to_prev_install'] > 1) | pd.isnull(installs['seconds_to_prev_install'])]

### Busco los installs que no se corresponden con un evento (un install deberia tener un evento en la tabla events para esa fecha). Para ellos genero un evento (774 eventos nuevos)

In [7]:
installs_with_events = pd.merge(events, installs, on=['ref_hash', 'date_seconds_level', 'application_id'])
installs_with_events['install_with_event'] = True
installs = pd.merge(installs, installs_with_events[['ref_hash', 'date_seconds_level', 'application_id', 'install_with_event']], on=['ref_hash', 'date_seconds_level', 'application_id'], how='left')
installs.head()
installs.loc[installs['install_with_event'] != True, 'install_with_event'] = False

In [8]:
#Creo el evento por cada install no correspondido (solo los installs implicitos tienen un evento correspondiente)
events = pd.concat([events, installs.loc[(installs['install_with_event'] == False) & (installs['implicit'] == True), ['ref_hash', 'created', 'application_id']]], ignore_index=True)

/home/maximiliano/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [9]:
events.loc[pd.isnull(events['date']), 'date'] = pd.to_datetime(events['created'])
events.drop(['created'], axis=1, inplace=True)

### Agrego columnas a events, clicks y a installs para poder clasificar despues

In [10]:
installs['install_day'] = installs['created'].dt.day
events['event_day'] = events['date'].dt.day
clicks['click_day'] = clicks['created'].dt.day
#auctions['auction_day'] = auctions['date'].dt.day

### Calculo la distancia de los clicks a la cruz de cierre de la impresion (basandonos en un heatmap realizado)

In [11]:
clicks['distanceToCloseButton'] = np.sqrt(abs(clicks['touchX_float'] - 0.95)**2 + abs(clicks['touchY_float'] - 0.05)**2)

# Creando el dataframe

### Separo los dataframe para obtener las distintas ventanas con las cuales entrenar

In [12]:
print(installs['created'].min(), installs['created'].max())
print(events['date'].min(), events['date'].max())
print(clicks['created'].min(), clicks['created'].max())

2019-04-18 00:00:01.560000 2019-04-26 23:59:58.788000
2019-04-18 00:00:00.027000 2019-04-26 23:59:59.881000
2019-04-12 00:00:01.981000 2019-04-26 23:59:22.065000


In [13]:
df_installs = {}
df_events = {}
df_clicks = {}
#df_auctions = {}
devices = {}
for i in range(18, 23):
    windowStartDate = datetime.datetime(2019, 4, i)
    windowEndDate = datetime.datetime(2019, 4, i + 3)
    df_installs[i - 18] = installs.loc[(installs['created'] >= windowStartDate) & (windowEndDate >= installs['created'])]
    df_events[i - 18] = events.loc[(events['date'] >= windowStartDate) & (windowEndDate >= events['date'])]
    df_clicks[i - 18] = clicks.loc[(clicks['created'] >= windowStartDate) & (windowEndDate >= clicks['created'])]
    #df_auctions[i - 18] = auctions.loc[(auctions['date'] >= windowStartDate) & (windowEndDate >= auctions['date'])]
    
    #Creo un dataframe de dispositivos con todos los dispositivos que tienen eventos para esa ventana
    devices[i - 18] = pd.DataFrame(df_events[i - 18]['ref_hash'].unique())
    
    #Agrego los segundos a la proxima conversion (desde el inicio de la ventana)
    next_window_installs = installs.loc[(installs['created'] >= datetime.datetime(2019, 4, i + 3)) & (datetime.datetime(2019, 4, i + 5) >= installs['created'])]
    devices[i - 18].columns = ['ref_hash']
    devices[i - 18] = pd.merge(devices[i - 18], next_window_installs.groupby('ref_hash')['created'].min().reset_index(), on='ref_hash', how='left')
    devices[i - 18].rename(columns = {'created':'seconds_to_conversion'}, inplace=True)
    devices[i - 18]['seconds_to_conversion'] = (devices[i - 18]['seconds_to_conversion'] - windowStartDate)/np.timedelta64(1, 's')
    
    #Completo los seconds_to_conversion de los que no convirtieron con el valor correspondiente a 3 dias en segundos
    devices[i - 18].loc[pd.isnull(devices[i - 18]['seconds_to_conversion']), 'seconds_to_conversion'] = 259200
    
    #Agrego los segundos a la proxima subasta (desde el inicio de la ventana)
    #next_window_auctions = auctions.loc[(auctions['date'] >= datetime.datetime(2019, 4, i + 3)) & (datetime.datetime(2019, 4, i + 5) >= auctions['date'])]
    #devices[i - 18] = pd.merge(devices[i - 18], next_window_auctions.groupby('device_id')['date'].min().reset_index(), left_on='ref_hash', right_on='device_id', how='left')
    #devices[i - 18].rename(columns = {'date':'seconds_to_auction'}, inplace=True)
    #devices[i - 18]['seconds_to_auction'] = (devices[i - 18]['seconds_to_auction'] - windowStartDate)/np.timedelta64(1, 's')
    
    #Completo los seconds_to_auction de los que no tuvieron subastas con el valor correspondiente a 3 dias en segundos
    #devices[i - 18].loc[pd.isnull(devices[i - 18]['seconds_to_auction']), 'seconds_to_auction'] = 259200

## Agrego features para cada dataframe de devices

In [14]:
for i in range(0, 5):
    #Porcentaje de installs implicitos y no implicitos en el periodo + total installs en el periodo
    installs_by_implicit = pd.pivot_table(df_installs[i], index='ref_hash', columns='implicit', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_implicit.drop(installs_by_implicit.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_implicit.columns = ['ref_hash', 'installs_not_implicit', 'installs_implicit', 'total_installs']
    installs_by_implicit['ref_hash'] = installs_by_implicit['ref_hash'].astype('int64')
    installs_by_implicit['installs_not_implicit_percent'] = installs_by_implicit['installs_not_implicit']/installs_by_implicit['total_installs']
    installs_by_implicit['installs_implicit_percent'] = installs_by_implicit['installs_implicit']/installs_by_implicit['total_installs']
    devices[i] = pd.merge(devices[i], installs_by_implicit[['ref_hash', 'installs_not_implicit_percent', 'installs_implicit_percent']], on='ref_hash', how='left')
    
    #Porcentaje de installs por dia en el periodo
    installs_by_day = pd.pivot_table(df_installs[i], index='ref_hash', columns='install_day', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    installs_by_day.drop(installs_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    installs_by_day.columns = ['ref_hash', 'installs_day_1', 'installs_day_2', 'installs_day_3', 'total_installs']
    installs_by_day['ref_hash'] = installs_by_day['ref_hash'].astype('int64')
    installs_by_day['installs_day_1_percent'] = installs_by_day['installs_day_1']/installs_by_day['total_installs']
    installs_by_day['installs_day_2_percent'] = installs_by_day['installs_day_2']/installs_by_day['total_installs']
    installs_by_day['installs_day_3_percent'] = installs_by_day['installs_day_3']/installs_by_day['total_installs']
    installs_by_day['total_installs_normalized'] = installs_by_day['total_installs']/installs_by_day['total_installs'].max()
    devices[i] = pd.merge(devices[i], installs_by_day, on='ref_hash', how='left')
    del installs_by_day
    
    #Porcentaje de eventos por dia en el periodo + total eventos
    events_by_day = pd.pivot_table(df_events[i], index='ref_hash', columns='event_day', values='date', aggfunc='count', fill_value=0, margins=True).reset_index()
    events_by_day.drop(events_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    events_by_day.columns = ['ref_hash', 'events_day_1', 'events_day_2', 'events_day_3', 'total_events']
    events_by_day['ref_hash'] = events_by_day['ref_hash'].astype('int64')
    events_by_day['events_day_1_percent'] = events_by_day['events_day_1']/events_by_day['total_events']
    events_by_day['events_day_2_percent'] = events_by_day['events_day_2']/events_by_day['total_events']
    events_by_day['events_day_3_percent'] = events_by_day['events_day_3']/events_by_day['total_events']
    events_by_day['total_events_normalized'] = events_by_day['total_events']/events_by_day['total_events'].max()
    devices[i] = pd.merge(devices[i], events_by_day, on='ref_hash', how='left')
    del events_by_day
    
    #Calculo el porcentaje de eventos hechos a traves de wifi (esto puede influir en un futuro install)
    events_by_wifi = pd.pivot_table(df_events[i], index='ref_hash', columns='wifi', values='date', aggfunc='count', fill_value=0).reset_index()
    events_by_wifi['events_on_wifi_percent'] = events_by_wifi[True]/(events_by_wifi[True] + events_by_wifi[False])
    devices[i] = pd.merge(devices[i], events_by_wifi[['ref_hash', 'events_on_wifi_percent']], on='ref_hash', how='left')
    del events_by_wifi
    
    #Porcentaje de clicks por dia en el periodo
    clicks_by_day = pd.pivot_table(df_clicks[i], index='ref_hash', columns='click_day', values='created', aggfunc='count', fill_value=0, margins=True).reset_index()
    clicks_by_day.drop(clicks_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    clicks_by_day.columns = ['ref_hash', 'clicks_day_1', 'clicks_day_2', 'clicks_day_3', 'total_clicks']
    clicks_by_day['ref_hash'] = clicks_by_day['ref_hash'].astype('int64')
    clicks_by_day['clicks_day_1_percent'] = clicks_by_day['clicks_day_1']/clicks_by_day['total_clicks']
    clicks_by_day['clicks_day_2_percent'] = clicks_by_day['clicks_day_2']/clicks_by_day['total_clicks']
    clicks_by_day['clicks_day_3_percent'] = clicks_by_day['clicks_day_3']/clicks_by_day['total_clicks']
    clicks_by_day['total_clicks_normalized'] = clicks_by_day['total_clicks']/clicks_by_day['total_clicks'].max()
    devices[i] = pd.merge(devices[i], clicks_by_day, on='ref_hash', how='left')
    del clicks_by_day
    
    #Trabajando con la posicion de los clicks (promedio de distancia al boton de cerrar publicidad y )
    devices[i] = pd.merge(devices[i], df_clicks[0].loc[(~np.isnan(df_clicks[0]['touchX_float'])) & (~np.isnan(df_clicks[0]['touchY_float']))].groupby('ref_hash')['distanceToCloseButton'].mean().reset_index(), on='ref_hash', how='left')
    devices[i].rename(columns = {'distanceToCloseButton':'mean_distance_to_close_button'}, inplace=True)
    
    #Completo la distancia al boton de cerrar con el promedio de la columna
    devices[i]['mean_distance_to_close_button'].fillna(devices[i]['mean_distance_to_close_button'].mean(), inplace=True)
    
    #Completo los nan con 0 todas las columnas restantes
    devices[i].fillna(0, inplace=True)
    
    #Porcentaje de subastas por dia en el periodo + total subastas
    #auctions_by_day = pd.pivot_table(df_auctions[i], index='device_id', columns='auction_day', values='date', aggfunc='count', fill_value=0, margins=True).reset_index()
    #auctions_by_day.drop(auctions_by_day.tail(1).index, inplace=True) #Elimino el margin creado por pivot_table
    #auctions_by_day.columns = ['ref_hash', 'auctions_day_1', 'auctions_day_2', 'auctions_day_3', 'total_auctions']
    #auctions_by_day['ref_hash'] = auctions_by_day['ref_hash'].astype('int64')
    #auctions_by_day['auctions_day_1_percent'] = auctions_by_day['auctions_day_1']/auctions_by_day['total_auctions']
    #auctions_by_day['auctions_day_2_percent'] = auctions_by_day['auctions_day_2']/auctions_by_day['total_auctions']
    #auctions_by_day['auctions_day_3_percent'] = auctions_by_day['auctions_day_3']/auctions_by_day['total_auctions']
    #devices[i] = pd.merge(devices[i], auctions_by_day, on='ref_hash', how='left')
    #del auctions_by_day

In [15]:
#Libero memoria
import gc
gc.collect()

786

## Agrego informacion del dispositivo. Esto puede tardar

In [17]:
def get_most_frequent_value(s):
    return np.nan if s.isnull().all() else s.value_counts().index[0]

for i in range(0, 5):
    #Agrego los valores mas frecuentes para device_model, device_brand, ref_type y device_os en events
    info_events = df_events[i].groupby('ref_hash')
    devices_info_events = info_events['device_model'].apply(get_most_frequent_value).reset_index()
    devices_info_events['device_brand'] = info_events['device_brand'].apply(get_most_frequent_value).reset_index()['device_brand']
    devices_info_events['ref_type'] = info_events['ref_type'].apply(get_most_frequent_value).reset_index()['ref_type']
    devices_info_events['device_os'] = info_events['device_os'].apply(get_most_frequent_value).reset_index()['device_os']
    devices[i] = pd.merge(devices[i], devices_info_events, on='ref_hash', how='left')
    del info_events, devices_info_events
    
    #Agrego los valores mas frecuentes para os_minor y os_major en clicks
    info_clicks = df_clicks[i].groupby('ref_hash')
    devices_info_clicks = info_clicks['os_minor'].apply(get_most_frequent_value).reset_index()
    devices_info_clicks['os_major'] = info_clicks['os_major'].apply(get_most_frequent_value).reset_index()['os_major']
    del info_clicks, devices_info_clicks

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
corr = devices[0][devices[0].columns].corr()
plt.figure(figsize = (30,20))
sns.heatmap(corr, annot = True, linewidths=0.2, fmt=".2f")

In [16]:
for i in range(0, 5):
    devices[i].to_csv('devices_featured{}.csv'.format(i), header=True, index=None)